# Concepts avancés Python
Nous allons maintenant voir quelques facilités fournies par le langage python
- introspection des objets
- itérateurs
- générateurs
- gestion des classes et objets : classes abstraites, attributs/méthodes de classes 
- décorateurs
- héritage multiple

## Introspection 
En python on peut avoir accès à certaines définitions internes des objets que l'on manipule. Par exemple : 

In [1]:
class Test:
    """une classe de test"""
    def __init__(self,x,y):
        self.one = x
        self.two = y
        
    def add(self,z):
        """on refait l'addition"""
        self.one += z
        self.rwo +=z

# n'est pas éxécuté si le script est importé par un autre script
if __name__=="__main__":
    a = Test(1,2)
    print(a.__dict__)
    print(a.__module__)
    print(a.__doc__)
    print(a.__class__)
    help(a)

{'one': 1, 'two': 2}
__main__
une classe de test
<class '__main__.Test'>
Help on Test in module __main__ object:

class Test(builtins.object)
 |  Test(x, y)
 |  
 |  une classe de test
 |  
 |  Methods defined here:
 |  
 |  __init__(self, x, y)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  add(self, z)
 |      on refait l'addition
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [2]:
print(__name__)

__main__


In [3]:
print(dir(a))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add', 'one', 'two']


## Itérateurs
On a vu que l'on pouvait itérer directement sur des contenants comme des listes

    for x in [1,3,5]:
        print(x)

Supposons que l'on définisse un objet qui contient lui aussi un ensemble de valeurs quelconque

In [3]:
class Equipe:   
    def __init__(self,nom,membres):
        self.nom = nom
        self.membres = membres
        
team = Equipe("Preum",["Adam","Eve"]) 
# on  peut itérer sur les membres
for x in team.membres:
    print(x)    

Adam
Eve


mais on voudrait directement itérer sur l'objet, qui est une collection de membres
le seul champ sur lequel on veut pouvoir itérer

In [4]:
for x in team:
    print(x)

TypeError: 'Equipe' object is not iterable

In [5]:
class Equipe:
    
    def __init__(self,nom,membres):
        self.nom = nom
        self.membres = membres
    
    # on peut définir un itérateur sur la classe. 
    def __iter__(self):
        return iter(self.membres)

team = Equipe("Preum",["Adam","Eve"])
for x in team:
    print(x)      

Adam
Eve


**Mais comment est défini l'itérateur sur la liste ?**  
Il suffit que l'objet ait une méthode `__next__()`, qui est déjà défini sur les listes.  
`__next__()` doit soulever une exception StopIteration quand il est censé s'arrêter (pas obligatoire...).   
On peut le définir soi-même sur un objet

## Exercice
Définir un itérateur qui parcours une liste à l'envers. 

In [7]:
 class Reverse:
    """itérateur pour parcourir un conteneur à l'envers"""
    def __init__(self, data):
        self.data = data
        self.index = len(data)
   
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

    def reset(self):
        self.index = len(self.data)
    

for x in Reverse([1,2,3]):
    print(x,end=" ") 
print()
    
a = Reverse([1,2,3])
for x in Reverse(range(10)):
    print(x,end=" ")

3 2 1 
9 8 7 6 5 4 3 2 1 0 

In [2]:
a = iter([1,2,3])
print(next(a))
print(a.__next__())
print(a.__next__())
print(a.__next__())
print(a.__next__())


1
2
3


StopIteration: 

## Générateurs
Un générateur est un moyen plus simple de définir des itérateurs.  
Le mot clef `yield` dans une fonction suffit à la définir la fonction comme générateur 

In [2]:
def natural(n):
    i = 0
    while i<n:
        yield i
        i = i + 1

#gen = natural(3)
for x in natural(3):
    print(x,end=" ")

0 1 2 

Un générateur permet de générer au fur et à mesure des besoins des valeurs à énumérer, sans tout stocker explicitement.  
La syntaxe cache en fait les choses suivantes: 
- la définition de l'initialisation de l'itérateur
- la définition de ce qu'il renvoie à chaque étape (next)
- l'exception quand on sort de ce qu'il doit énumérer

La fin de l'itération est optionnelle !

In [3]:
def natural0():
    i = 1
    while True:
        yield i
        i = i + 1
    
gen = natural0()
print(next(gen), gen.__next__(), gen.__next__(), gen.__next__())

1 2 3 4


Un générateur définit implicitement un itérateur, à condition de s'arrêter. 

In [4]:
for i in natural(100):
    print(i,end=" ") 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

En fait, `natural` fait la même chose que la fonction `range`
On peut définir un parcours d'un conteneur "à l'avance" 

### Exemple

In [5]:
from sys import getsizeof
# là on crée une liste à chaque fois
a1 = [2*x for x in range(10000)] 
print(getsizeof(a1))
a1 = [x-3 for x in a1]
print(getsizeof(a1))

l = [x for x in range(100) if x>50]

87624
87624


In [6]:
# ou bien ...
a2 = (2*x for x    in range(10000))
print(a2)
print(getsizeof(a2))
a2 = (x-3 for   x in a2) 
print(a2)
print(getsizeof(a2))

# plus simple
a2 = (y-3 for y in (2*x for x in range(100)))

for x in a2:
    print(x,end=" ") 
    
a2.__next__()

<generator object <genexpr> at 0x0000014DB730FC48>
120
<generator object <genexpr> at 0x0000014DB730FEC8>
120
-3 -1 1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45 47 49 51 53 55 57 59 61 63 65 67 69 71 73 75 77 79 81 83 85 87 89 91 93 95 97 99 101 103 105 107 109 111 113 115 117 119 121 123 125 127 129 131 133 135 137 139 141 143 145 147 149 151 153 155 157 159 161 163 165 167 169 171 173 175 177 179 181 183 185 187 189 191 193 195 

StopIteration: 

In [7]:
print(list(a2)[:100])

[]


In [8]:
for (i,x) in enumerate(["a","d","e"]):
    print(i,x) 

0 a
1 d
2 e


In [9]:
[x+10 for x in range(10) if x % 2==0]

[10, 12, 14, 16, 18]

In [10]:
# On peut appliquer les méthodes de générateur
from random import randint
maliste = (randint(-1000,1000) for i in range(10000))

print(next(i for (i,x) in enumerate(maliste) if x==0))

# 3 générateurs et complexité espace en O(1) ! 

# question: qu'a renvoyé cette expression ? 

1656


In [11]:
#--> l'indice du premier zero

In [12]:
# attention quand même : 
print(sum(next(i for (i,x) in enumerate(maliste) if x==0) \
          for j in range(1000))/1000.)

RuntimeError: generator raised StopIteration

Revenons à notre classe ;  on aurait pu écrire l'itérateur nous mêmes:

In [13]:
class Equipe:
    def __init__(self,nom,membres):
        self.nom = nom
        self. membres = membres
    
    def __iter__(self):
        for x in self.membres:
            yield x
            
team = Equipe("Preum",["Adam","Eve"])
for x in team:
    print(x)      
    
# on peut gérer nous mêmes la fin de l'itération   
a = iter(team)
next(a),next(a)
try: 
    next(a)
except StopIteration:
    print("ayé")

Adam
Eve
ayé


A quoi ça sert ? à contrôler plus finement ce qui se passe  
[Aussi et surtout: économie de mémoire]

In [14]:
class Equipe:
    def __init__(self,nom,membres):
        self.nom = nom
        self.membres = membres
        self.bannis = set()
    
    def __iter__(self):
        for x in self.membres:
            if x not in self.bannis:
                yield x

team = Equipe("Preum",["Adam","Eve","Cain","Abel"])
team.bannis.add("Cain")

for x in team:
    print(x)   
    
    

Adam
Eve
Abel


### Exercice : Analyse d'ADN 
- codon = 3 'lettres, fin seulement {TAA, TAG, TGA}; début TTG ATG GTG
- 'code génétique': acide aminé codé par chaque codon

Faire un générateur qui analyse l'ADN codon par codon, en commençant au premier début valide et en s'arrêtant à la première fin.   
Faire un générateur qui envoie toutes les acides aminés codées sur un brin (bonus: il peut y avoir plusieurs début/fin)
  
      

In [14]:
# codon -> acide aminé 
code = {'TCA':'S', # Sérine
 'TCC':'S', # Sérine
  'TCG':'S', # Sérine
  'TCT':'S', # Sérine
  'TTC':'F', # Phénylalanine
  'TTT':'F', # Phénylalanine
  'TTA':'L', # Leucine
  'TTG':'L', # Leucine
  'TAC':'Y', # Tyrosine
  'TAT':'Y', # Tyrosine
  'TAA':'*', # Codon Stop
  'TAG':'*', # Codon Stop
  'TGC':'C', # Cystéine
  'TGT':'C', # Cystéine
  'TGA':'*', # Codon Stop
  'TGG':'W', # Tryptophane
  'CTA':'L', # Leucine
  'CTC':'L', # Leucine
  'CTG':'L', # Leucine
  'CTT':'L', # Leucine
  'CCA':'P', # Proline
  'CCC':'P', # Proline
  'CCG':'P', # Proline
  'CCT':'P', # Proline
  'CAC':'H', # Histidine
  'CAT':'H', # Histidine
  'CAA':'Q', # Glutamine
  'CAG':'Q', # Glutamine
  'CGA':'R', # Arginine
  'CGC':'R', # Arginine
  'CGG':'R', # Arginine
  'CGT':'R', # Arginine
  'ATA':'I', # Isoleucine
  'ATC':'I', # Isoleucine
  'ATT':'I', # Isoleucine
  'ATG':'M', # Méthionine
  'ACA':'T', # Thréonine
  'ACC':'T', # Thréonine
  'ACG':'T', # Thréonine
  'ACT':'T', # Thréonine
  'AAC':'N', # Asparagine
  'AAT':'N', # Asparagine
  'AAA':'K', # Lysine
  'AAG':'K', # Lysine
  'AGC':'S', # Sérine
  'AGT':'S', # Sérine
  'AGA':'R', # Arginine
  'AGG':'R', # Arginine
  'GTA':'V', # Valine
  'GTC':'V', # Valine
  'GTG':'V', # Valine
  'GTT':'V', # Valine
  'GCA':'A', # Alanine
  'GCC':'A', # Alanine
  'GCG':'A', # Alanine
  'GCT':'A', # Alanine
  'GAC':'D', # Acide Aspartique
  'GAT':'D', # Acide Aspartique
  'GAA':'E', # Acide Glutamique
  'GAG':'E', # Acide Glutamique
  'GGA':'G', # Glycine
  'GGC':'G', # Glycine
  'GGG':'G', # Glycine
  'GGT':'G', # Glycine 
  }

adn = "GGTGGGGTTGCAGCCCTGAAGCTTACACCCTGA"

debut = {"TTG","GTG","ATG"}
end = {"TTA","TAG","TTA"}

["G","C","..."]

['G', 'C', '...']

In [15]:
def sequencage(adn):
    # trouver le début
    i = 0
    while adn[i:i+3] not in debut:#
        i = i + 1
    # cracher les codons 
    i = i + 3
    while i <= len(adn)-3:
        codon = adn[i:i+3]
        i = i + 3
        yield codon


#for codon in sequencage(adn):
#    print(codon)


def decodage(adn,dico_code):
    for codon in sequencage(adn):
        proteine = dico_code[codon]
        if proteine != "*":
            yield proteine
        else:
            # ou bien return None
            #raise StopIteration
            return None
        
for proteine in decodage(adn,code):
    print(proteine)

G
L
Q
P


## Retour sur les définitions de classes
- attributs de classes
- classes abstraite

On peut avoir des attributs de classes, partagés par toutes les instances

In [16]:
class Bidon:
    # attribut de classe
    general = [1,2,3]
    
    def __init__(self,x):
        self.particulier = x
        

a = Bidon(5)
b = Bidon(6)

print(a.general, b.general)
a.general[0] = a.particulier
print(b.general)
print(a.general is b.general)
print(a.particulier,b.particulier)

[1, 2, 3] [1, 2, 3]
[5, 2, 3]
True
5 6


**Mais comment faire pour avoir une fonction partagée par toutes les instances sans l'instance elle-même ?**  
Impossible comme les attributs car le premier argument est toujours l'objet créé (self)  
-> On a besoin d'autre chose (plus tard)

## Classes Abstraites

Rappel: une classe abstraite est une classe qui définit une API susceptible de recevoir plusieurs implémentations différentes (dans des sous-classes), et qui laisse donc certaines méthodes non implémentées.  
En Java, on parle d'`Interfaces`.  
Il est facile d'implémenter une telle classe en utilisant l'exception `NotImplementError`

In [16]:
class AbstractMaClasse:
    def __init__(self,x):
        self.base = x
    def a_method(self,x):
        return x*2
    def another(self,x,y):
        raise NotImplementedError("appel à une classe abstraite")

In [17]:
class MaClasse(AbstractMaClasse):
    # note: la liste d'argument n'est bien sur pas figée
    def another(self,x,y):
        return self.base + x*y
    
a = AbstractMaClasse(1)
print("objet a créé ")
b = MaClasse(2)
print("objet b créé")
print(b.another(2,3))
print(a.another(2,3))

objet a créé 
objet b créé
8


NotImplementedError: appel à une classe abstraite

Un inconvénient de cette méthode est qu'il n'y a aucun contrôle que les sous-classes définissent bien toutes les méthodes au moment de la création d'une instance. 

In [18]:
class Oops(AbstractMaClasse):
    pass

a = Oops(1) # ne devrait pas marcher !
print("----- instance créée")
a.another(3,4) 

----- instance créée


NotImplementedError: appel à une classe abstraite

## Décorateurs
On a vu que l'on pouvait facilement définir des fonctions avec des fonctions en arguments ou même en résultat.  
Les fonctions sont des objets comme les autres (ou presque).  
Python fournit de plus une syntaxe spéciale pour faciliter les transformations de fonctions: les décorateurs.  
Ce mécanisme va permettre de traiter les problèmes mentionnés auparavant sur la définition des objets: méthode statique, méthode de classe, attributs protégés pratiques.

### Exemple
Faisons une fonction qui aide à déclarer des fonctions "obsolètes" (en donnant un avertissement, mais en laissant le programme continuer)

In [15]:
def obsolete(func):
    def new_func(*y,**opt):
        print("!! attention, appel à fonction obsolete:",func.__name__)
        return func(*y,**opt)
    return new_func

def mafonction(x):
    return x*2

mafonction = obsolete(mafonction)


mafonction(12)

!! attention, appel à fonction obsolete: mafonction


24

**Inconvénient**  
Ne marche que si la fonction a un seul argument.

Avec des arguments quelconques cette fois :

In [7]:
def obsolete(func):
    def new_func(*args,**kwargs):
        print("!! attention, appel à fonction obsolete:",func.__name__)
        return func(*args,**kwargs)
    return new_func

def mafonction(x,y):
    return x**2+y**2

mafonction = obsolete(mafonction)
mafonction(12,12)

!! attention, appel à fonction obsolete: mafonction


288

Python fournit en fait une syntaxe plus pratique

In [17]:
def obsolete(func):
    def new_func(*args,**kwargs):
        print("!! attention, appel à fonction obsolete:",func.__name__)
        return func(*args,**kwargs)
    return new_func

@obsolete
def mafonction(x,y):
    return x**2+y**2

mafonction(12,12)

!! attention, appel à fonction obsolete: mafonction


288

**Avantages des décorateurs**  
- plus concis -> plus clair !
- ne sépare pas la définition de la fonction de ses modifications
- permet de les empiler de façon lisible
- autorise les arguments

**Un outil d'abstraction très puissant !**  
Avec un argument -> revient à empiler un autre "emballage" (wrapper) par dessus le premier

In [18]:
def obsolete(date):
    def obsolete_decorator(func):
        def new_func(*args,**kwargs):
            print("!! attention, appel à fonction",
                  func.__name__,"obsolete depuis",date)
            return func(*args,**kwargs)
        return new_func
    return obsolete_decorator
    
@obsolete(2005)
def mafonction(x,y):
    return x**2+y**2

@obsolete(1980)
def oldfonction(x):
    return x*2

print(mafonction(12,12))
print(oldfonction(15))


!! attention, appel à fonction mafonction obsolete depuis 2005
288
!! attention, appel à fonction oldfonction obsolete depuis 1980
30


Trois décorateurs utiles sont prédéfinis: 
- `@property` - définit un attribut avec des accès/modifications contrôlées. 
- `@staticmethod` - méthode d'une class partagée par toutes les instances
- `@classmethod` - méthode de classe -> utiles pour héritage, permet de définir des méthodes statiques sans mettre le nom de classe explicitement et donc peut être repris par les sous-classes telle quelle 
- `@abc.abstractmethod` permet de définir des méthodes de classes abstraites (module abc)

### Attributs protégés
Et si l'on voulait protéger des attributs et garder une syntaxe simple pour l'accès et la modification ?  
-> On utilise le décorateur `@property`

Reprenons l'exemple de la classe Date

In [19]:
# lourd et redondant ....
class Date:
    
    def __init__(self,jour,mois,annee):
        if jour>=1 and jour<=31:
            self.jour = jour
        else:
            self.jour = None
        if mois>=1 and mois<=12:
            self.mois = mois
        else:
            self.mois = None
        if annee!=0:
            self.annee = annee
        else:
            self.annee = None
            
    def set_jour(self,new):
        if new>=1 and new<=31:
            self.jour = new
            return True
        else:
            self.jour = None
            return False
        
    def get_jour(self):
        return self.jour
    
    # idem mois / annee

# serait déjà mieux avec attributs protégés: self.__jour, etc
# et en utilisant set/get dans l'__init__
# mais toujours un peu lourd

In [20]:
# avec property
class Date:
    
    def __init__(self,jour,mois,annee):
        self.jour = jour
        self.mois = mois
        self.annee = annee
        
    @property
    def jour(self):
        return self.__jour
    
    @jour.setter
    def jour(self,new):
        if new>=1 and new<=31:
            self.__jour = new
        else:
            self.__jour = None
        

d = Date(28,1,2019)
#d.jour = 45
print(d.jour)

28


In [21]:
# note: l'attribut est bien protégé: 
print(d.__jour)

AttributeError: 'Date' object has no attribute '__jour'

In [22]:
d._Date__jour

28

## Méthode statique: 
- methode d'une classe qui ne dépend pas d'une instance 
- si on déclare normalement, on a une copie de la fonction à chaque instance

### Exemple
Supposons qu'on redéfinisse une classe complexe, et qu'on veut une méthode qui donne les racines enièmes de 1, i.e.

   $$\exp(i 2k\pi/n) = cos(2k\pi/n) + i\cdot sin(2k\pi/n) $$

In [7]:
from math import cos,sin, pi

class Complexe:
    def __init__(self,r,i):
        self.reel = r
        self.imaginaire = i
        
    def __repr__(self):
        return "%f + i*(%f)"%(self.reel,self.imaginaire)
    
    def racine_unite(self,n,k):
        return  Complexe(cos(2*k*pi/n),sin(2*k*pi/n))

a = Complexe(1,1)
b = Complexe(-1,3)
print(a.racine_unite(3,1))
a.racine_unite is b.racine_unite

-0.500000 + i*(0.866025)


False

En fait l'instance est inutile ici, on voudrait juste avoir les arguments n et k;  
On définit alors la méthode comme "statique" pour ne pas avoir besoin d'instance

In [8]:
from math import cos,sin, pi

class Complexe:
    def __init__(self,r,i):
        self.reel = r
        self.imaginaire = i
        
    def __repr__(self):
        return "%f + i*(%f)"%(self.reel,self.imaginaire)
    
    @staticmethod
    def racine_unite(n,k):
        return Complexe(cos(2.0*k*pi/n),sin(2.0*k*pi/n))

a = Complexe(1,1)
b = Complexe(-1,3)
print(Complexe.racine_unite(3,1))
a.racine_unite is b.racine_unite

-0.500000 + i*(0.866025)


True

### Exercice
Faire une méthode pour créer un complexe à partir d'une forme polaire ($\theta$, r)

### Méthodes de classe
Imaginons maintenant qu'on veuille définir un nouveau conteneur, que l'on peut initialiser avec une liste, mais aussi à partir des éléments d'un dictionnaire.  
On a deux solutions : 

In [13]:
class Table:
    def __init__(self,items):
            self.contenu = items
    
    def from_dict1(self,dico):
            self.contenu = dico.values()
            
    def from_dict2(self,dico):
            return Table(dico.values())

Pas très élégant, car il faut soit initialiser deux fois une instance ou bien créer une instance pour créer une autre instance

In [14]:
a = Table([])
a.from_dict1({1:2,3:4})
# ou bien 
c = a.from_dict2({5:6,7:8})

Déjà mieux avec une méthode statique

In [15]:
class Table:
     def __init__(self,items):
            self.contenu = items
    
     @staticmethod
     def from_dict(dico):
            return Table(dico.values())
            
a = Table.from_dict({}) 
print(a)

**Problème**  
Si on veut maintenant faire une sous classe de `Table`, on doit tout réécrire si on veut garder la cohérence des constructeurs (from_dict faisant explicitement appel à la surclasse `Table`)

In [16]:
# une table qui contient plus d'information
class AutreTable(Table):
     def __init__(self,items):
            self.contenu = items
            self.nb = len(items)
    
     @staticmethod
     def from_dict(dico):
            return AutreTable(dico.values())
            
 

Ne serait-il pas mieux d'avoir une méthode de classe à la place ? Du coup rien à réécrire.

In [28]:
class Table:
     def __init__(self,items):
            self.contenu = items
    
     @classmethod
     def from_dict(cls,dico):
            return cls(dico.values())

class AutreTable(Table):
     def __init__(self,items):
            self.contenu = items
            self.nb = len(items)
    

a = AutreTable.from_dict({1:2})
a.__class__

__main__.AutreTable

### Exercices
Définir des décorateurs pour 
- garder en cache des résultats de fonction ("memoisation")
- compter les appels de certaines fonctions
- répéter une fonction jusqu'à atteindre une condition [exam 2017_2018]
   
**Indice**  
Définir le décorateur comme objet et redéfinir la méthode `__call__`



In [23]:
# exemple de méthode __call__
class Bidon:
    def __init__(self):
        self.ct = 0
    
    def __call__(self,x):
        self.ct += 1
        return x**2
    
call_me = Bidon()
print(call_me(3))
print(call_me(6))
print(call_me.ct)

9
36
2


In [24]:
# avec fonction à un seul argument
class memoise:
    
    def __init__(self,func):
        self.func = func
        self.memoire = {}
    
    def __call__(self,*x):
        if x in self.memoire:
            return self.memoire[x]
        else:
            val = self.func(*x)
            self.memoire[x] = val
            return val

# decorateur équivalent à : 
#fibo = memoise(fibo)
#puis appel normal fibo(50)

@memoise
def fibo(n):
    if n<2:
        return 1
    else:
        return fibo(n-1)+fibo(n-2)

fibo(400)

284812298108489611757988937681460995615380088782304890986477195645969271404032323901

In [25]:
# version plus générale
class memoise:
    
    def __init__(self,f):
        self.func = f
        self.cache = {}
        
    def __call__(self,*args):
        if args in self.cache:
            return self.cache[args]
        else:
            val = self.func(*args)
            self.cache[args] = val 
            return val
        
@memoise
def fibo(n):
    if n<2:
        return 1
    else:
        return fibo(n-1)+fibo(n-2)
    
fibo(100)

573147844013817084101

In [27]:
class comptage:
    counts = {}
    
    def __init__(self,f):
        self.func = f
        self.name = f.__name__
        self.counts[self.name] = 0
    
    def __call__(self,*args):
        self.counts[self.name] = self.counts[self.name] + 1
        return self.func(*args)
    
    def __repr__(self):
        return self.func.__doc__
    
    @staticmethod
    def resultats():
        return comptage.counts

@comptage
def add(x,y):
    """addition"""
    return x+y

@comptage 
def sub(x,y):
    """soustraction"""
    return x-y

add(3,sub(4,5))
add(5,3)
add(6,7)
print(comptage.resultats())
print(add)

{'add': 3, 'sub': 1}
addition


In [38]:
add(6,8)
comptage.resultats()

{'add': 4, 'sub': 1}

In [28]:
# répétition
def repeat(times):
    def wrapper(func):
            def newfunc(*args):
                n = 0
                success = False
                while not(success) and n < times:
                    success = func(*args)
                    n = n + 1
                return success
            return newfunc
    return wrapper

from random import random

# version de base
def test(seuil):
    a = random()
    print(a)
    return a>seuil

print(test(0.7))
print('-'*5)
# version transformée
@repeat(5)
def test(seuil):
    a = random()
    print(a)
    return a>seuil

print(test(0.7))

0.9789289707491233
True
-----
0.41390938153502754
0.9604165533647303
True


## Classes abstraites

On a vu que définir des classes abstraites seulement avec des exceptions `NotImplemtend`.  
Ceci est aussi permit par le module `abc`, qui définit le décorateur de méthode abstraite

In [29]:
import abc 

class AbstractMaClass(object):
    __metaclass__  = abc.ABCMeta

    def __init__(self):
        pass
    
    @abc.abstractmethod
    def another(self):
        pass

a = AbstractMaClass()
b = AbstractMaClass()
print(a,b)

<__main__.AbstractMaClass object at 0x0000014DB73CF208> <__main__.AbstractMaClass object at 0x0000014DB73CF308>


In [30]:
class Vrai(AbstractMaClass):
    pass

c = Vrai()
print(c.another())  

None


## Héritage multiple
On peut faire hériter une classe de plusieurs classes pour récupérer tous les traits et attributs en les combinant.  
La syntaxe est simple ... la sémantique un peu moins.  
Imaginons un héritage multiple avec des classes ayant des méthodes de même nom ...  

In [31]:
class A:
    pass
    def m(self):
        print("appel de A.m")

class B(A):
    pass
    def m1(self):
        print("appel de B.m")
    
class C(A):
    def m(self):
        print("appel de C.m")

# héritage multiple 
class D(B,C):
    pass

a = D()
a.m()

appel de C.m


Python résoud les héritages de fonction en commençant par la première superclasse si possible, puis les autres en suivant. 

**Quand faire de l'héritage multiple ?**
- jamais ? On peut toujours s'en tirer en composant les classes B et C
        class D: 
            def __init__(self):
                self._b = B()
                self._c = C()
- en ne mélangeant pas hiérarchie et utilisation. dans l'exemple ci-dessus soit B soit C ne devrait pas 
    être transmis à D, en tout cas pas avec des parents directs

**Exemple**  
Imaginons une classe polygone, avec des sous-classes rectangle, losange, carré.  
Il est tentant de considérer le carré comme héritant à la fois de losange et rectangle.  
En pratique il suffit d'hériter d'une seule classe pour "faire le job".  